In [1]:
import os
import random
import itertools
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
pwd

'/tf/image-classification/src/trainer'

In [3]:
base_dir = '../../../dataset/dist-unbalanced/'

train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

train_accepted_dir = os.path.join(train_dir, 'accepted')

train_rejected_dir = os.path.join(train_dir, 'rejected')

valid_accepted_dir = os.path.join(valid_dir, 'accepted')

valid_rejected_dir = os.path.join(valid_dir, 'rejected')

In [4]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [5]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 64, class_mode = 'binary', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(valid_dir,  batch_size = 64, class_mode = 'binary', target_size = (224, 224))

Found 9778 images belonging to 2 classes.
Found 1090 images belonging to 2 classes.


In [6]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

In [7]:
for layer in base_model.layers:
    layer.trainable = False

In [8]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(32, activation='relu'))
base_model.add(Dense(1, activation='sigmoid'))

In [9]:
checkpoint_filepath = './models/resnet100.h5'

mc = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

es = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0,
    patience=50,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0)

In [10]:
base_model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001),
                   loss = 'binary_crossentropy', 
                   metrics = ['accuracy'])

In [12]:
train_generator.class_indices

{'accepted': 0, 'rejected': 1}

In [13]:
resnet_history = base_model.fit(train_generator, 
                                validation_data = validation_generator, 
                                steps_per_epoch = 100, 
                                epochs = 100,
                                callbacks=[mc, es])

Epoch 1/100
100/100 [==============================] - 45s 447ms/step - loss: 0.8073 - accuracy: 0.6118 - val_loss: 1.0836 - val_accuracy: 0.4954
Epoch 2/100
100/100 [==============================] - 45s 448ms/step - loss: 0.7322 - accuracy: 0.6278 - val_loss: 1.0705 - val_accuracy: 0.4963
Epoch 3/100
100/100 [==============================] - 45s 442ms/step - loss: 0.7112 - accuracy: 0.6353 - val_loss: 0.9420 - val_accuracy: 0.4954
Epoch 4/100
100/100 [==============================] - 44s 438ms/step - loss: 0.6812 - accuracy: 0.6391 - val_loss: 0.8755 - val_accuracy: 0.5202
Epoch 5/100
100/100 [==============================] - 44s 438ms/step - loss: 0.6727 - accuracy: 0.6420 - val_loss: 0.7539 - val_accuracy: 0.5835
Epoch 6/100
100/100 [==============================] - 43s 431ms/step - loss: 0.6569 - accuracy: 0.6455 - val_loss: 0.7014 - val_accuracy: 0.6110
Epoch 7/100
100/100 [==============================] - 43s 424ms/step - loss: 0.6397 - accuracy: 0.6544 - val_loss: 0.6714 -

Epoch 57/100
100/100 [==============================] - 44s 436ms/step - loss: 0.4468 - accuracy: 0.7917 - val_loss: 0.5024 - val_accuracy: 0.7532
Epoch 58/100
100/100 [==============================] - 43s 431ms/step - loss: 0.4322 - accuracy: 0.7958 - val_loss: 0.5024 - val_accuracy: 0.7569
Epoch 59/100
100/100 [==============================] - 43s 429ms/step - loss: 0.4315 - accuracy: 0.7959 - val_loss: 0.4980 - val_accuracy: 0.7532
Epoch 60/100
100/100 [==============================] - 43s 422ms/step - loss: 0.4296 - accuracy: 0.7966 - val_loss: 0.4970 - val_accuracy: 0.7560
Epoch 61/100
100/100 [==============================] - 43s 426ms/step - loss: 0.4323 - accuracy: 0.7949 - val_loss: 0.4938 - val_accuracy: 0.7587
Epoch 62/100
100/100 [==============================] - 43s 427ms/step - loss: 0.4163 - accuracy: 0.8063 - val_loss: 0.4940 - val_accuracy: 0.7587
Epoch 63/100
100/100 [==============================] - 44s 439ms/step - loss: 0.4212 - accuracy: 0.8006 - val_loss: 0

In [ ]:
base_model.save('500epoch.h5')

In [ ]:
pwd

In [ ]:
base_path = '../../../dataset/dist-unbalanced'
SHAPE = (224,224,3)
batch_size = 256

In [ ]:
img = np.random.choice(os.listdir(os.path.join(base_path , "test")))
img_path = os.path.join(base_path, "test", img)
img = cv2.imread(img_path)
img = cv2.resize(img, SHAPE[:2])
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGRA2RGB))
plt.show()

In [ ]:
img = keras.preprocessing.image.load_img(
    img_path, target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = float(predictions[0])
print(f"This image is {100 * (1 - score):.2f}% cat and {100 * score:.2f}% dog.")


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

In [14]:
img = image.load_img(img_path, target_size=(224, 224))

NameError: name 'image' is not defined

In [ ]:
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)

In [ ]:
img_batch.shape

In [ ]:
img_preprocessed = preprocess_input(img_batch)

In [ ]:
img_preprocessed.shape

In [ ]:
model = tf.keras.models.load_model('./models/100epoch.h5')

In [ ]:
model.predict(img_batch)